<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Recognition of hand-written digits with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/raw/master/scikit-learn/hand-written-digits-recognition/images/numbers_banner-04.png" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to retrieve data from the IBM Data Science Experience Community, create a predictive model, and start scoring new data. This notebook introduces commands for retrieving, cleaning, and exploring data as well as pipeline creation, model training, model persistance to the Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.5 and scikit-learn 0.17 package.

You will use a toy dataset available from the [scikit-learn](http://scikit-learn.org/stable/index.html) Web site, **sklearn.datasets.load_digits**, which contains hand-written digits images. Use the toy dataset to recognize hand-written digits. Refer to the [Citations](#citations) for information about use of these images and scikit-learn modules.

## Learning goals

The learning goals of this notebook are:

-  Load a toy dataset from scikit-learn.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create a scikit-learn machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in the Watson Machine Learning repository.
-  Deploy a model for online scoring using the Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the matplotlib package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create a scikit-learn model](#model)
4.	[Persist the model](#persistence)
5.	[Predict locally and visualize](#visualization)
6.	[Deploy and score in the cloud](#scoring)
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup task:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data from scikit-learn toy datasets and perform a basic exploration.

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install sklearn --user <BR>

In [ ]:
from sklearn import datasets

digits = datasets.load_digits()

The data set consists of 8x8 pixel images of hand-written digits. In the following step you will plot the first 5 images using the **matplotlib** library.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

images_number = 5
images_and_labels = list(zip(digits.images, digits.target))

for i, (image, label) in enumerate(images_and_labels[:images_number]):
    plt.subplot(2, images_number, i + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('%i' % label)

Let's display the first-digit data and label it using the `data` and `target` methods.

In [ ]:
print(digits.data[0])

In [ ]:
digits.target[0]

In next step you will count all the samples.

In [ ]:
samples_count = len(digits.images)
print("Number of samples: " + str(samples_count))

<a id="model"></a>
## 3. Create a scikit-learn machine learning model

In this section you will learn how to prepare data, create a scikit-learn machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into train, test, and score data sets.

In [ ]:
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]


print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))


As you can see our data has been successfully split into the following data sets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.
-  The score data set will be used for scoring in the cloud.

### 3.2: Create a pipeline and train a model

In this section you will create scikit-learn machine learning pipeline and then train the model.

In the first step you need to import the scikit-learn machine learning packages that will be needed in the subsequent steps.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

Standardize features by removing the mean and scaling to unit variance.

In [ ]:
scaler = preprocessing.StandardScaler()

Next, define estimators you want to use for classification. Support Vector Machines with radial basis function as kernel is used in the following example.

In [ ]:
clf = svm.SVC(kernel='rbf')

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [ ]:
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [ ]:
model = pipeline.fit(train_data, train_labels)

You can check your **model quality** now. To evaluate the model, use the **test data**.

In [ ]:
predicted = model.predict(test_data)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

You can tune your model now to achieve better accuracy. For the sake of simplicity, in this example the tuning section is omitted.

<a id="persistence"></a>
## 4. Persist the model

In this section you will learn how to store your pipeline and model in the Watson Machine Learning repository by using python client libraries.

First, you must import client libraries.

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
from repository.mlrepository import MetaProps, MetaNames

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [ ]:
service_path = 'https://ibm-watson-ml.mybluemix.net'
username = '***'
password = '***'
instance_id = '***'

**Tip**: The `service_path`, `username`, and `password` values can be found on the **Service Credentials** tab of the service instance that you created in Bluemix.

In [ ]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

Create the model artifact (abstraction layer).

In [ ]:
props = MetaProps({MetaNames.AUTHOR_NAME:"IBM", MetaNames.AUTHOR_EMAIL:"ibm@ibm.com"})

In [ ]:
model_artifact = MLRepositoryArtifact(model, name="Hand-written digits recognition", meta_props=props)

**Tip**: The `MLRepositoryArtifact` method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save the pipeline and model

In this subsection you will learn how to save the pipeline and model artifacts to your Watson Machine Learning instance.

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

Retrieve the saved model metadata from Watson Machine Learning.

**Tip**: Use the `meta.available_props()` method to get the list of available props.

In [ ]:
saved_model.meta.available_props()

In [ ]:
print("modelType: " + saved_model.meta.prop("modelType"))
print("runtime: " + saved_model.meta.prop("runtime"))
print("creationTime: " + str(saved_model.meta.prop("creationTime")))
print("modelVersionHref: " + saved_model.meta.prop("modelVersionHref"))

**Tip**: The **modelVersionHref** variable is our model unique indentifier in the Watson Machine Learning repository.

### 4.2: Load the model

In this subsection you will learn how to load back the saved model from a specified instance of Watson Machine Learning.

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print the model name to make sure that model has been loaded correctly.

In [ ]:
print(loadedModelArtifact.name)
print(saved_model.uid)

As you can see the name is correct. You have already learned how to save and load the model from the Watson Machine Learning repository.

<a id="visualization"></a>
## 5. Predict locally and visualize

In this section you will learn how to score test data by using the loaded model and visualize the prediction results by using the `plotly` package.

### 5.1: Make local prediction using previously loaded model and score data

In this subsection you will score the `predict_data` data set.

In [ ]:
predictions = loadedModelArtifact.model_instance().predict(score_data)

To print the prediction results, run the following code:

In [ ]:
print(predictions)

### 5.2: Sample visualization of data with the `matplotlib` package

In [ ]:
images_and_predictions = list(zip(digits.images[int(0.9*samples_count): ], predictions))
for i, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, i + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

<a id="scoring"></a>
## 6. Deploy and score in the cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [ ]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

### 6.1: Create the model deployment

Retrieve the `published_models` URL from the instance details.

#### Get instance details

In [ ]:
endpoint_instance = service_path + "/v3/wml_instances/" + instance_id
header = {'Content-Type': 'application/json', 'Authorization': mltoken}

response_get_instance = requests.get(endpoint_instance, headers=header)
print(response_get_instance)
print(response_get_instance.text)

In [ ]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')

print(endpoint_published_models)

Execute the following sample code that uses the `published_models` endpoint to get the deployments URL.

#### Get the list of published models

In [ ]:
header = {'Content-Type': 'application/json', 'Authorization': mltoken}
response_get = requests.get(endpoint_published_models, headers=header)

print(response_get)
print(response_get.text)

#### Get the published model deployment URL

In [ ]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]

print(endpoint_deployments)

#### Create an online deployment for the published model

In [ ]:
payload_online = {"name": "Hand written digits recognition", "description": "Hand Written Digits Deployment", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print(response_online)
print(response_online.text)

In [ ]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')

print(scoring_url)

 Now, you can send (POST) new scoring records (new data) for which you would like to get predictions. To do that, execute the following sample code: 

In [ ]:
digit = list(digits.data[1])
digit2 = list(digits.data[2])

In [ ]:
payload_scoring = {"values": [digit, digit2]}
print(payload_scoring)

In [ ]:
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)

print(response_scoring.text)

As we can see we predict that hand-written digits are: 1 and 2.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2) for more samples, tutorials, documentation, how-tos, and blog posts. 

<a id="citations"></a>
### Citations

[Scikit-learn: Machine Learning in Python](http://jmlr.csail.mit.edu/papers/v12/pedregosa11a.html), Pedregosa et al., JMLR 12, pp. 2825-2830, 2011

[API design for machine learning software: experiences from the scikit-learn project](https://arxiv.org/abs/1309.0238), Buitinck et al., 2013.


### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.